--- Day 22: Wizard Simulator 20XX ---

Little Henry Case decides that defeating bosses with swords and stuff is boring. Now he's playing the game with a wizard. Of course, he gets stuck on another boss and needs your help again.

In this version, combat still proceeds with the player and the boss taking alternating turns. The player still goes first. Now, however, you don't get any equipment; instead, you must choose one of your spells to cast. The first character at or below 0 hit points loses.

Since you're a wizard, you don't get to wear armor, and you can't attack normally. However, since you do magic damage, your opponent's armor is ignored, and so the boss effectively has zero armor as well. As before, if armor (from a spell, in this case) would reduce damage below 1, it becomes 1 instead - that is, the boss' attacks always deal at least 1 damage.

On each of your turns, you must select one of your spells to cast. If you cannot afford to cast any spell, you lose. Spells cost mana; you start with 500 mana, but have no maximum limit. You must have enough mana to cast a spell, and its cost is immediately deducted when you cast it. Your spells are Magic Missile, Drain, Shield, Poison, and Recharge.

    Magic Missile costs 53 mana. It instantly does 4 damage.
    Drain costs 73 mana. It instantly does 2 damage and heals you for 2 hit points.
    Shield costs 113 mana. It starts an effect that lasts for 6 turns. While it is active, your armor is increased by 7.
    Poison costs 173 mana. It starts an effect that lasts for 6 turns. At the start of each turn while it is active, it deals the boss 3 damage.
    Recharge costs 229 mana. It starts an effect that lasts for 5 turns. At the start of each turn while it is active, it gives you 101 new mana.

Effects all work the same way. Effects apply at the start of both the player's turns and the boss' turns. Effects are created with a timer (the number of turns they last); at the start of each turn, after they apply any effect they have, their timer is decreased by one. If this decreases the timer to zero, the effect ends. You cannot cast a spell that would start an effect which is already active. However, effects can be started on the same turn they end.

For example, suppose the player has 10 hit points and 250 mana, and that the boss has 13 hit points and 8 damage:

-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 13 hit points
Player casts Poison.

-- Boss turn --
- Player has 10 hit points, 0 armor, 77 mana
- Boss has 13 hit points
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 damage.

-- Player turn --
- Player has 2 hit points, 0 armor, 77 mana
- Boss has 10 hit points
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 2 hit points, 0 armor, 24 mana
- Boss has 3 hit points
Poison deals 3 damage. This kills the boss, and the player wins.

Now, suppose the same initial conditions, except that the boss has 14 hit points instead:

-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 14 hit points
Player casts Recharge.

-- Boss turn --
- Player has 10 hit points, 0 armor, 21 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 4.
Boss attacks for 8 damage!

-- Player turn --
- Player has 2 hit points, 0 armor, 122 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 3.
Player casts Shield, increasing armor by 7.

-- Boss turn --
- Player has 2 hit points, 7 armor, 110 mana
- Boss has 14 hit points
Shield's timer is now 5.
Recharge provides 101 mana; its timer is now 2.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 1 hit point, 7 armor, 211 mana
- Boss has 14 hit points
Shield's timer is now 4.
Recharge provides 101 mana; its timer is now 1.
Player casts Drain, dealing 2 damage, and healing 2 hit points.

-- Boss turn --
- Player has 3 hit points, 7 armor, 239 mana
- Boss has 12 hit points
Shield's timer is now 3.
Recharge provides 101 mana; its timer is now 0.
Recharge wears off.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 2 hit points, 7 armor, 340 mana
- Boss has 12 hit points
Shield's timer is now 2.
Player casts Poison.

-- Boss turn --
- Player has 2 hit points, 7 armor, 167 mana
- Boss has 12 hit points
Shield's timer is now 1.
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 1 hit point, 7 armor, 167 mana
- Boss has 9 hit points
Shield's timer is now 0.
Shield wears off, decreasing armor by 7.
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 1 hit point, 0 armor, 114 mana
- Boss has 2 hit points
Poison deals 3 damage. This kills the boss, and the player wins.

You start with 50 hit points and 500 mana points. The boss's actual stats are in your puzzle input. What is the least amount of mana you can spend and still win the fight? (Do not include mana recharge effects as "spending" negative mana.)


In [1]:
filepath = "..\\data\\input_day_22.txt"
test1 = "..\\test\\test22_1.txt"

In [2]:
# first we import our files
def read_input(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    return lines

In [3]:
def get_boss(lines):
    for line in lines:
        
        if "Hit Points" in line:
            hp = int(line.strip().split()[-1])
        else:
            damage = int(line.strip().split()[-1])
    return {"hp": hp, "dmg": damage, "poison": 0}

In [4]:
def get_player():
    return {"hp": 50, "mana": 500, "armor": 0, "shield": 0, "recharge": 0}

In [5]:
def set_poison(boss):
    boss["poison"] = 6
    return boss

In [6]:
def set_shield(player):
    player["shield"] = 6
    return player

In [7]:
def set_recharge(player):
    player["recharge"] = 5
    return player

In [8]:
def update_turn_player(player):
    player["armor"] = 0
    
    # check for shield and update
    if player["shield"] > 0:
        player["armor"] += 7
        player["shield"] -= 1
        
    # check for recharge
    if player["recharge"] > 0:
        player["mana"] += 101
        player["recharge"] -= 1
        
    return player

In [9]:
def update_turn_boss(boss):
    
    if boss["poison"] > 0:
        boss["hp"] -= 3
        boss["poison"] -= 1
    
    return boss

In [10]:
def determine_winner(player, boss):
    
    # if the boss wins
    if player["hp"] <= 0:
        return 2
    
    # if the player wins
    if boss["hp"] <= 0:
        return 1
    
    # nobody wins
    return 0 

In [11]:
def cast_spell(player, boss, spell):
    
    costs = {"Magic Missile": 53, "Drain": 73, "Shield": 113,
             "Poison": 173, "Recharge": 229}
    cost = 0    
    # player casts magic missile
    if spell == "Magic Missile":
        player["mana"] -= costs[spell]
        if player["mana"] <= 0:
            return True, 0, 0, 0
        boss["hp"] -= 4
        
    if spell == "Drain":
        player["mana"] -= costs[spell]
        if player["mana"] <= 0:
            return True, 0, 0, 0
        boss["hp"] -= 2
        player["hp"] += 2
    
    if spell == "Shield":
        player["mana"] -= costs[spell]
        if player["mana"] <= 0 or player["shield"]>0:
            return True, 0, 0, 0
        player = set_shield(player)
        
    if spell == "Poison":
        player["mana"] -= costs[spell]
        if player["mana"] <= 0 or boss["poison"]>0:
            return True, 0, 0, 0
        boss = set_poison(boss)
        
    if spell == "Recharge":
        player["mana"] -= costs[spell]
        if player["mana"] <= 0 or player["recharge"]>0:
            return True, 0, 0, 0
        player = set_recharge(player)
        
    return False, player, boss, costs[spell]

In [12]:
def boss_damage(player, boss):
    
    player["hp"] -= max((boss["dmg"]-player["armor"]), 1)
    return player, boss

In [38]:
def simulate_fight(player, spells, boss):
    
    # keep track of the mana spent
    mana_spent = 0
    
    for spell in spells:
        # each round of combat consists of two phases the player phase and the boss phase        
        # player phase
        #print(spell, player, boss)
        # update shield and recharge
        player = update_turn_player(player)
        
        # player casts his spell
        no_mana, player, boss, cost = cast_spell(player, boss, spell)
        if no_mana:
            #print("no mana")
            return False, cost
        mana_spent += cost
        
        result = determine_winner(player, boss)
        if result == 2:
            return False, cost
        if result == 1:
            return True, cost
        
        # boss phase
        # update health for poison 
        boss = update_turn_boss(boss)
        
        result = determine_winner(player, boss)
        if result == 2:
            return False, cost
        if result == 1:
            return True, cost
        
        
        player, boss = boss_damage(player, boss)
        
        result = determine_winner(player, boss)
        if result == 2:
            return False, cost
        if result == 1:
            return True, cost
    print(spell, player, boss)
    return False, cost
        

In [41]:
def day22a(filepath):
    
    from itertools import combinations_with_replacement
    
    boss_stats = read_input(filepath)
    spells = ["Magic Missile", "Drain", "Shield", "Poison", "Recharge"]
    combinations = list(combinations_with_replacement(spells, 100))
    costs = []
    
    for combi in combinations:
        #print(combi)
        
        boss = get_boss(boss_stats)
        player = get_player()
        
        player_wins, cost = simulate_fight(player, combi, boss)
        if player_wins:
            costs.append(cost)
            
    print(costs)

In [42]:
day22a(filepath)

[]


In [16]:
lines = read_input(filepath)


for line in lines:
    print(line)
    if "Hit Points" in line:
        hp = int(line.strip().split()[-1])
    else:
        damage = int(line.strip().split()[-1])
print({"hp": hp, "dmg": damage, "poison": 0}) 

Hit Points: 51

Damage: 9

{'hp': 51, 'dmg': 9, 'poison': 0}
